In [1]:
import pandas as pd
import numpy as np

In [2]:
def process_dataframe(df, away_team):
    # Drop specified columns
    df = df.drop(['www.bulibox.de', 'Unnamed: 5'], axis=1, errors='ignore')
    
    # Rename columns
    df = df.rename(columns={'Unnamed: 1': 'date', 
                            'Unnamed: 2':'matchday_nr', 
                            'Markiere Verein: ': 'home_team_name', 
                            away_team: 'away_team_name', 
                            'Unnamed: 6': 'hist_home_team_win', 
                            'Unnamed: 7': 'hist_draw', 
                            'Unnamed: 8': 'hist_away_team_win', 
                            'Unnamed: 9': 'hist_goal_dif'})
    
    # Drop first 3 rows
    df = df.drop(df.index[:3])
    
    # Drop last 10 rows
    df = df.drop(df.tail(10).index)
    
    # Convert 'matchday_nr', 'hist_home_team_win', 'hist_draw', 'hist_away_team_win' to integer
    cols_to_convert = ['matchday_nr', 'hist_home_team_win', 'hist_draw', 'hist_away_team_win']
    for col in cols_to_convert:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    
    return df

In [3]:
def calculate_historical_goal_difference(df, column_name):
    def calc_diff(x):
        try:
            if isinstance(x, str):  # Check if x is a string
                return int(x.split(':')[0]) - int(x.split(':')[1])
            else:
                return np.nan  # If not a string, return NaN
        except ValueError:
            return np.nan  # return NaN for rows where conversion to int fails

    df[column_name] = df[column_name].apply(calc_diff)
    return df

In [4]:
def extract_year(df, date_column):
    # Convert the date column to strings
    df[date_column] = df[date_column].astype(str)

    # Extract the year from the date column
    df['season'] = df[date_column].str.split('\D').apply(lambda x: [i for i in x if len(i) == 4][0] if [i for i in x if len(i) == 4] else None)

    # Convert the 'year' column to integers
    df['season'] = df['season'].astype(float).astype('Int64')
    
    # Drop the date column
    df = df.drop(columns=[date_column])
    
    return df

# Plan 23/24

In [5]:
df16 = pd.read_excel('data/plan_23_24.xls')
df16

,www.bulibox.de,Unnamed: 1,Unnamed: 2,Markiere Verein:,1. FC Heidenheim 1846,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Spielplan 1. Bundesliga 2023/2024,NaN,NaN,NaN,NaN,Direkter Vergleich\n(Alle 1.+2. BuLi-Spiele),NaN,NaN,NaN
2,NaN,Datum,Spieltag,Heimmannschaft,Gastmannschaft,NaN,S,U,N,Tore
3,NaN,2023-08-18 20:30:00,1,SV Werder Bremen,FC Bayern München,NaN,26,26,60,128:230
4,NaN,19./20.08.2023,1,VfB Stuttgart,VfL Bochum 1848,NaN,37,20,15,126:87
...,...,...,...,...,...,...,...,...,...,...
314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,NaN,Unter diesem Link hast Du die Möglichkeit den ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,NaN,sowie die einzelnen Spielpläne der Mannschafte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
away_team = input("Please enter the name of the away team: ")  # Get user input for away team
df16 = process_dataframe(df16, away_team)
df16

Please enter the name of the away team: 1. FC Heidenheim 1846


,date,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif
3,2023-08-18 20:30:00,1,SV Werder Bremen,FC Bayern München,26,26,60,128:230
4,19./20.08.2023,1,VfB Stuttgart,VfL Bochum 1848,37,20,15,126:87
5,19./20.08.2023,1,Borussia Dortmund,1. FC Köln,40,25,29,162:134
6,19./20.08.2023,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,12:7
7,19./20.08.2023,1,Bayer 04 Leverkusen,RB Leipzig,3,4,7,18:22
...,...,...,...,...,...,...,...,...
304,2024-05-18 15:30:00,34,Bayer 04 Leverkusen,FC Augsburg,15,7,2,53:22
305,2024-05-18 15:30:00,34,VfL Wolfsburg,1. FSV Mainz 05,19,15,10,75:55
306,2024-05-18 15:30:00,34,1. FC Union Berlin,Sport-Club Freiburg,5,3,4,17:20
307,2024-05-18 15:30:00,34,TSG Hoffenheim,FC Bayern München,4,7,19,28:70


In [7]:
df16 = calculate_historical_goal_difference(df16, 'hist_goal_dif')
df16

,date,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif
3,2023-08-18 20:30:00,1,SV Werder Bremen,FC Bayern München,26,26,60,-102.0
4,19./20.08.2023,1,VfB Stuttgart,VfL Bochum 1848,37,20,15,39.0
5,19./20.08.2023,1,Borussia Dortmund,1. FC Köln,40,25,29,28.0
6,19./20.08.2023,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,5.0
7,19./20.08.2023,1,Bayer 04 Leverkusen,RB Leipzig,3,4,7,-4.0
...,...,...,...,...,...,...,...,...
304,2024-05-18 15:30:00,34,Bayer 04 Leverkusen,FC Augsburg,15,7,2,31.0
305,2024-05-18 15:30:00,34,VfL Wolfsburg,1. FSV Mainz 05,19,15,10,20.0
306,2024-05-18 15:30:00,34,1. FC Union Berlin,Sport-Club Freiburg,5,3,4,-3.0
307,2024-05-18 15:30:00,34,TSG Hoffenheim,FC Bayern München,4,7,19,-42.0


In [8]:
df16 = extract_year(df16, 'date')
df16

,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif,season
3,1,SV Werder Bremen,FC Bayern München,26,26,60,-102.0,2023
4,1,VfB Stuttgart,VfL Bochum 1848,37,20,15,39.0,2023
5,1,Borussia Dortmund,1. FC Köln,40,25,29,28.0,2023
6,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,5.0,2023
7,1,Bayer 04 Leverkusen,RB Leipzig,3,4,7,-4.0,2023
...,...,...,...,...,...,...,...,...
304,34,Bayer 04 Leverkusen,FC Augsburg,15,7,2,31.0,2024
305,34,VfL Wolfsburg,1. FSV Mainz 05,19,15,10,20.0,2024
306,34,1. FC Union Berlin,Sport-Club Freiburg,5,3,4,-3.0,2024
307,34,TSG Hoffenheim,FC Bayern München,4,7,19,-42.0,2024


In [9]:
df16['home_team_name'].unique()

array(['SV Werder Bremen', 'VfB Stuttgart', 'Borussia Dortmund',
       'Eintracht Frankfurt', 'Bayer 04 Leverkusen', 'VfL Wolfsburg',
       '1. FC Union Berlin', 'FC Augsburg', 'TSG Hoffenheim',
       'FC Bayern München', 'Borussia Mönchengladbach', '1. FC Köln',
       'VfL Bochum 1848', 'Sport-Club Freiburg', '1. FSV Mainz 05',
       'SV Darmstadt 98', '1. FC Heidenheim 1846', 'RB Leipzig'],
      dtype=object)

In [10]:
df16['away_team_name'].unique()

array(['FC Bayern München', 'VfL Bochum 1848', '1. FC Köln',
       'SV Darmstadt 98', 'RB Leipzig', '1. FC Heidenheim 1846',
       '1. FSV Mainz 05', 'Borussia Mönchengladbach',
       'Sport-Club Freiburg', 'FC Augsburg', 'Bayer 04 Leverkusen',
       'VfL Wolfsburg', 'Borussia Dortmund', 'SV Werder Bremen',
       'Eintracht Frankfurt', '1. FC Union Berlin', 'TSG Hoffenheim',
       'VfB Stuttgart'], dtype=object)

In [11]:
# Define a mapping of current names to new names
name_mapping = {
    '1899 Hoffenheim': 'TSG 1899 Hoffenheim',
    'TSG Hoffenheim': 'TSG 1899 Hoffenheim',
    'Bayer 04 Leverkusen': 'Bayer Leverkusen',
    'Bayern München': 'FC Bayern München',
    'DSC Arminia Bielefeld': 'Arminia Bielefeld',
    'FC Augsburg 1907': 'FC Augsburg',
    'FSV Mainz 05': '1. FSV Mainz 05',
    'Hertha BSC Berlin': 'Hertha BSC',
    'SV Werder Bremen': 'Werder Bremen',
    'Schalke 04': 'FC Schalke 04',
    'Sport-Club Freiburg': 'SC Freiburg',
    'VfL Bochum 1848': 'VfL Bochum'
}

# Replace the home team names
df16['home_team_name'] = df16['home_team_name'].replace(name_mapping)

# Do the same for the away team names
df16['away_team_name'] = df16['away_team_name'].replace(name_mapping)

In [12]:
df16['home_team_name'].unique()

array(['Werder Bremen', 'VfB Stuttgart', 'Borussia Dortmund',
       'Eintracht Frankfurt', 'Bayer Leverkusen', 'VfL Wolfsburg',
       '1. FC Union Berlin', 'FC Augsburg', 'TSG 1899 Hoffenheim',
       'FC Bayern München', 'Borussia Mönchengladbach', '1. FC Köln',
       'VfL Bochum', 'SC Freiburg', '1. FSV Mainz 05', 'SV Darmstadt 98',
       '1. FC Heidenheim 1846', 'RB Leipzig'], dtype=object)

In [13]:
df16['away_team_name'].unique()

array(['FC Bayern München', 'VfL Bochum', '1. FC Köln', 'SV Darmstadt 98',
       'RB Leipzig', '1. FC Heidenheim 1846', '1. FSV Mainz 05',
       'Borussia Mönchengladbach', 'SC Freiburg', 'FC Augsburg',
       'Bayer Leverkusen', 'VfL Wolfsburg', 'Borussia Dortmund',
       'Werder Bremen', 'Eintracht Frankfurt', '1. FC Union Berlin',
       'TSG 1899 Hoffenheim', 'VfB Stuttgart'], dtype=object)

In [14]:
data = pd.read_csv('data.csv')
data

,league_name,season,location,matchday_nr,home_team_id,home_team_name,away_team_id,away_team_name,goals_home,goals_away,draw,win_home,win_away
0,1. Fussball-Bundesliga 2005/2006,2005,München,1,40,FC Bayern München,87,Borussia Mönchengladbach,3,0,0.0,1.0,0.0
1,1. Fussball-Bundesliga 2005/2006,2005,Köln,1,65,1. FC Köln,81,1. FSV Mainz 05,1,0,0.0,1.0,0.0
2,1. Fussball-Bundesliga 2005/2006,2005,Duisburg,1,107,MSV Duisburg,16,VfB Stuttgart,1,1,1.0,0.0,0.0
3,1. Fussball-Bundesliga 2005/2006,2005,Hamburg,1,100,Hamburger SV,79,1. FC Nürnberg,3,0,0.0,1.0,0.0
4,1. Fussball-Bundesliga 2005/2006,2005,Wolfsburg,1,131,VfL Wolfsburg,7,Borussia Dortmund,2,2,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,1. Fußball-Bundesliga 2022/2023,2022,Köln,34,65,1. FC Köln,40,FC Bayern München,1,2,0.0,0.0,1.0
5504,1. Fußball-Bundesliga 2022/2023,2022,Leipzig,34,1635,RB Leipzig,9,FC Schalke 04,4,2,0.0,1.0,0.0
5505,1. Fußball-Bundesliga 2022/2023,2022,Stuttgart,34,16,VfB Stuttgart,175,TSG 1899 Hoffenheim,1,1,1.0,0.0,0.0
5506,1. Fußball-Bundesliga 2022/2023,2022,Berlin,34,80,1. FC Union Berlin,134,Werder Bremen,1,0,0.0,1.0,0.0


In [15]:
# Display the unique pairs of 'home_team_id' and 'home_team_name'
distinct_pairs = data[['home_team_id', 'home_team_name']].drop_duplicates()
distinct_pairs

,home_team_id,home_team_name
0,40,FC Bayern München
1,65,1. FC Köln
2,107,MSV Duisburg
3,100,Hamburger SV
4,131,VfL Wolfsburg
5,134,Werder Bremen
6,55,Hannover 96
7,9,FC Schalke 04
8,91,Eintracht Frankfurt
9,6,Bayer Leverkusen


In [16]:
# Display the unique pairs of 'away_team_id' and 'away_team_name'
distinct_pairs = data[['away_team_id', 'away_team_name']].drop_duplicates()
distinct_pairs

,away_team_id,away_team_name
0,87,Borussia Mönchengladbach
1,81,1. FSV Mainz 05
2,16,VfB Stuttgart
3,79,1. FC Nürnberg
4,7,Borussia Dortmund
5,83,Arminia Bielefeld
6,54,Hertha BSC
7,76,1. FC Kaiserslautern
8,6,Bayer Leverkusen
9,40,FC Bayern München


In [17]:
# Remove duplicates from the 'data' DataFrame based on 'home_team_name'
data = data.drop_duplicates(subset='home_team_name', keep='last')

# Merge the 'home_team_id' column from 'data' into 'df16' based on the 'home_team_name' column
df16 = df16.merge(data[['home_team_name', 'home_team_id']], on='home_team_name', how='left')
df16

,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif,season,home_team_id
0,1,Werder Bremen,FC Bayern München,26,26,60,-102.0,2023,134.0
1,1,VfB Stuttgart,VfL Bochum,37,20,15,39.0,2023,16.0
2,1,Borussia Dortmund,1. FC Köln,40,25,29,28.0,2023,7.0
3,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,5.0,2023,91.0
4,1,Bayer Leverkusen,RB Leipzig,3,4,7,-4.0,2023,6.0
...,...,...,...,...,...,...,...,...,...
301,34,Bayer Leverkusen,FC Augsburg,15,7,2,31.0,2024,6.0
302,34,VfL Wolfsburg,1. FSV Mainz 05,19,15,10,20.0,2024,131.0
303,34,1. FC Union Berlin,SC Freiburg,5,3,4,-3.0,2024,80.0
304,34,TSG 1899 Hoffenheim,FC Bayern München,4,7,19,-42.0,2024,175.0


In [18]:
# Remove duplicates from the 'data' DataFrame based on 'home_team_name'
data = data.drop_duplicates(subset='away_team_name', keep='first')

# Merge the 'home_team_id' column from 'data' into 'df16' based on the 'home_team_name' column
df16 = df16.merge(data[['away_team_name', 'away_team_id']], on='away_team_name', how='left')
df16

,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif,season,home_team_id,away_team_id
0,1,Werder Bremen,FC Bayern München,26,26,60,-102.0,2023,134.0,40.0
1,1,VfB Stuttgart,VfL Bochum,37,20,15,39.0,2023,16.0,129.0
2,1,Borussia Dortmund,1. FC Köln,40,25,29,28.0,2023,7.0,65.0
3,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,5.0,2023,91.0,NaN
4,1,Bayer Leverkusen,RB Leipzig,3,4,7,-4.0,2023,6.0,1635.0
...,...,...,...,...,...,...,...,...,...,...
301,34,Bayer Leverkusen,FC Augsburg,15,7,2,31.0,2024,6.0,95.0
302,34,VfL Wolfsburg,1. FSV Mainz 05,19,15,10,20.0,2024,131.0,81.0
303,34,1. FC Union Berlin,SC Freiburg,5,3,4,-3.0,2024,80.0,112.0
304,34,TSG 1899 Hoffenheim,FC Bayern München,4,7,19,-42.0,2024,175.0,40.0


In [19]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df16)

,Total,Percent
away_team_id,34,11.111111
hist_home_team_win,18,5.882353
hist_draw,18,5.882353
hist_away_team_win,18,5.882353
hist_goal_dif,18,5.882353
home_team_id,17,5.555556
matchday_nr,0,0.000000
home_team_name,0,0.000000
away_team_name,0,0.000000
season,0,0.000000


In [20]:
nan_rows = df16[df16['away_team_id'].isnull()]
nan_rows

,matchday_nr,home_team_name,away_team_name,hist_home_team_win,hist_draw,hist_away_team_win,hist_goal_dif,season,home_team_id,away_team_id
3,1,Eintracht Frankfurt,SV Darmstadt 98,5,0,3,5.0,2023,91.0,NaN
5,1,VfL Wolfsburg,1. FC Heidenheim 1846,<NA>,<NA>,<NA>,NaN,2023,131.0,NaN
21,3,Borussia Dortmund,1. FC Heidenheim 1846,<NA>,<NA>,<NA>,NaN,2023,7.0,NaN
23,3,Bayer Leverkusen,SV Darmstadt 98,5,0,1,6.0,2023,6.0,NaN
38,5,VfB Stuttgart,SV Darmstadt 98,5,4,3,9.0,2023,16.0,NaN
42,5,Bayer Leverkusen,1. FC Heidenheim 1846,<NA>,<NA>,<NA>,NaN,2023,6.0,NaN
59,7,Eintracht Frankfurt,1. FC Heidenheim 1846,<NA>,<NA>,<NA>,NaN,2023,91.0,NaN
61,7,FC Augsburg,SV Darmstadt 98,5,3,8,-6.0,2023,95.0,NaN
73,9,FC Bayern München,SV Darmstadt 98,7,1,0,13.0,2023,40.0,NaN
75,9,Borussia Mönchengladbach,1. FC Heidenheim 1846,<NA>,<NA>,<NA>,NaN,2023,87.0,NaN


Darmstadt 98 has an ID but Heidenheim not, since they have never been in the Bundesliga. 

In [21]:
# Assign value 118 to 'away_team_id' where 'away_team_name' is 'SV Darmstadt 98'
df16.loc[df16['away_team_name'] == 'SV Darmstadt 98', 'away_team_id'] = 118

In [22]:
# Dropping all the rows with 1. FC Heidenheim 1846 since we have no information about them
df16.dropna(inplace=True)

In [23]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df16)

,Total,Percent
matchday_nr,0,0.0
home_team_name,0,0.0
away_team_name,0,0.0
hist_home_team_win,0,0.0
hist_draw,0,0.0
hist_away_team_win,0,0.0
hist_goal_dif,0,0.0
season,0,0.0
home_team_id,0,0.0
away_team_id,0,0.0


In [24]:
df16.to_csv('future_game_plan_23_24.csv', index=False)